In [33]:
from joblib import Memory

memory = Memory(".cache", verbose=0)

In [34]:
from transformers import BertTokenizer, BertModel
import torch
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

@memory.cache
def get_tokenizer_model():
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    bert_model = BertModel.from_pretrained('bert-base-cased')
    sbert_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

    return bert_tokenizer, bert_model, sbert_model

bert_tokenizer, bert_model, sbert_model = get_tokenizer_model()

In [35]:
from pathlib import Path
import itertools
from natsort import natsorted

#@memory.cache
def read_problem_files(problem_folder, n=None):
    """
    reads ground truth files into dict
    :param truth_folder: path to folder holding ground truth files
    :return: dict of ground truth files with problem-id as key and file content as value
    """
    problems = []
    files = itertools.islice(natsorted(Path(problem_folder).glob('problem-*.txt')), n)
    for problem_file in files:
        # number = problem_file.name[len("problem-") : -len(".txt")]
        with open(problem_file, 'r', encoding="utf8") as fh:
            problems.append(fh.readlines())
    return problems

In [36]:
from evaluation.evaluator import read_ground_truth_files

@memory.cache
def cached_read_ground_truth(x):
    return read_ground_truth_files(x)

ground_truth = cached_read_ground_truth("pan21/train")

In [37]:
def get_embeddings(paragraph):
    # Tokenize the sentence
    tokens = bert_tokenizer.tokenize(paragraph)
    token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)

    # Convert token IDs to tensor
    token_tensor = torch.tensor([token_ids])
    segment_ids = [0] * len(token_ids)
    segment_tensor = torch.tensor([segment_ids])

    # Get BERT model output
    with torch.no_grad():
        outputs = bert_model(token_tensor, segment_tensor)

    # Extract word embeddings from BERT model output
    return outputs[0]

In [38]:
problems = read_problem_files("pan21/train", n=5)
problems[0]

["As stephelton said in the comments to your question, vector math is extremely important for pretty much any 2D or 3D game. However, physics knowledge isn't necessary for a lot of simple games. There are physics-like concepts you should understand a bit about, like collision, but you won't need calculus or physics classes for that as long as you keep it simple. A lot of things you may want to do can be simulated simply enough that players won't care much, like friction, or sliding, or gravity. A decent grasp of physics will likely help in many situations though.\n",
 'It is probably not required to know physics in details when you\'re doing a game, but it definitely helps, especially if there are some \'virtual reality\' features in your game. A game like "From Dust" (Eric Chahi) is essentially physics simulation gamified, while "Another World" only need high-precision capture of real-life motion (so and requires little to no actual understanding of what happens).\n',
 "It is very lik

In [39]:
#@memory.cache
# def get_max_length(n=None):
#     max_train = max([len(x) for y in read_problem_files("pan21/train", n=n) for x in y])
#     max_val = max([len(x) for y in read_problem_files("pan21/validation", n=n) for x in y])
#     return max([max_train, max_val])

In [40]:
import torch
import numpy as np

max_input_length = 512

def pad_paragraph(paragraph_embedding, desired_length):
    d1, d2, d3 = paragraph_embedding.shape
    # print(f"{paragraph_embedding.shape=}")

    target = torch.zeros(d1, desired_length, d3)
    # print(f"{target.shape=}")
    target[:, :d2, :] = paragraph_embedding

    return target


def get_paragraph_pairs(problem_text):
    # print(problem_text)
    paragraph_embeddings = [pad_paragraph(get_embeddings(para), max_input_length) for para in problem_text]
    # print(f"{[paras.shape for paras in paragraph_embeddings]=}")
    # print(f"{[x.shape for x in paragraph_embeddings]}")
    pairs = itertools.combinations(paragraph_embeddings, 2)
    return [torch.flatten(torch.stack(pair, dim=2), start_dim=1, end_dim=2) for pair in pairs]


In [41]:
from tqdm.notebook import tqdm

# #@memory.cache
def get_problem_embeddings(problems):
    return [get_paragraph_pairs(problem_text) for problem_text in tqdm(problems)]

In [42]:
# [get_paragraph_pairs(problem_text) for problem_text in tqdm(read_problem_files("pan21/train", n=5))][0][0].shape

In [43]:
problems_embed = get_problem_embeddings(problems)
for i in range(5):
    print(problems_embed[i][0][0].shape)
print(f"{problems_embed[0][0].shape=}")

  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1024, 768])
torch.Size([1024, 768])
torch.Size([1024, 768])
torch.Size([1024, 768])
torch.Size([1024, 768])
problems_embed[0][0].shape=torch.Size([1, 1024, 768])


In [44]:
def get_simple_ground_truth(ground_truth, problem_numbers):
    simple_ground_truth = []
    for num in problem_numbers:
        task_3_ground_truth = ground_truth[f"problem-{num}"]["paragraph-authors"]
        simple_ground_truth.append(task_3_ground_truth)
    return simple_ground_truth


simple_ground_truth = get_simple_ground_truth(ground_truth, range(1, len(problems_embed)))

In [14]:
def get_task_3_ground_truth(simple_ground_truth):
    task_gt = []
    for problem in simple_ground_truth:
        problem_gt = []
        for author1, author2 in itertools.combinations(problem, 2):
            problem_gt.append(int(author1 != author2))
        task_gt.append(problem_gt)
    return task_gt

task_3_ground_truth = get_task_3_ground_truth(simple_ground_truth)

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
# TODO: Invert the function get_task_3_ground_truth. Our model will output a bunch of binary labels which need to be converted to the task 3 ground truth format
# Ground truth format (gtf): [1, 2, 2, 2, 2, 3, 2, 2]
# Binary labels for comparisons (bl): [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]
# Each binary label is the result of comparing two paragraphs. 1 means there was an author change, 0 means there was no author change
# For example, bl[0], is the result of comparing gtf[0]=1 and gtf[1]=2. 1 != 2, therefore bl[0] = 1. bl[1]=1 is the result of gtf[0] == gtf[2] (1 == 2)
def get_simple_ground_truth_from_task_3(task_3_ground_truth):
    pass

simple_ground_truth == get_simple_ground_truth_from_task_3(task_3_ground_truth)

False

In [16]:
for s, t in zip(simple_ground_truth, task_3_ground_truth):
    print(f"{s}")
    print(f"{t}")

[1, 2, 2, 2, 2, 3, 2, 2]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]
[1, 2, 2, 2, 1, 2]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[1, 2, 2, 3, 3, 2]
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
# Add code to write out the embeddings (X) and ground truths (y) so we can train without having to rerun the preprocessing step

In [18]:
import numpy as np

def flatten_problems(problems_list):
    return [pair for problem in problems_list for pair in problem]

x_train = None
y_train = None

x_train = flatten_problems(get_problem_embeddings(read_problem_files("pan21/train", n=50)))
y_train = flatten_problems(get_task_3_ground_truth(get_simple_ground_truth(cached_read_ground_truth("pan21/train"), range(1, len(x_train)+1))))
x_val = flatten_problems(get_problem_embeddings(read_problem_files("pan21/validation", n=15)))
y_val = flatten_problems(get_task_3_ground_truth(get_simple_ground_truth(cached_read_ground_truth("pan21/validation"), range(1, len(x_val)+1))))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/991 [00:00<?, ?it/s]

  0%|          | 0/991 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

In [19]:
print(f"Num training examples: {len(x_train)=} {len(y_train)=}")

Num training examples: len(x_train)=991 len(y_train)=25205


In [26]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
x_test, y_test = x_val, y_val


In [31]:
get_task_3_ground_truth(get_simple_ground_truth(cached_read_ground_truth("pan21/train"), range(1, len(x_train)+1)))

  0%|          | 0/991 [00:00<?, ?it/s]

  0%|          | 0/991 [00:00<?, ?it/s]

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0],
 [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
 [1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1],
 [0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 1, 1, 0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed
 
# Code implementation of the RNN for sequence labeling
def create_rnn_model(vocab_size, num_labels, embedding_dim, lstm_units):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(TimeDistributed(Dense(num_labels, activation='softmax')))
    return model
 
# Example usage of the RNN model
vocab_size = 30000  # Replace with the actual size of the vocabulary
num_labels = 2  # Replace with the actual number of entity labels
embedding_dim = 768
lstm_units = 64
max_sequence_length = 35

max_input_length = get_max_length(n=len(x_train))
 
model = create_rnn_model(vocab_size, num_labels, embedding_dim, lstm_units)
model.build((1, max_input_length))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 2362, 768)         │    23,040,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 2362, 64)          │       213,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (1, 2362, 2)           │           130 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,253,378 (88.70 MB)

 Trainable params: 23,253,378 (88.70 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
from tensorflow.keras import optimizers, losses, metrics


model.compile(
    optimizer=optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[metrics.SparseCategoricalAccuracy()],
)


In [29]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 991
'y' sizes: 25205


In [33]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Evaluate on test data


ValueError: Unrecognized data type: x=[[tensor([[[ 0.3419,  0.0934,  0.1832,  ..., -0.4819,  0.0700,  0.2659],
         [ 0.2685,  0.0025,  0.3448,  ..., -0.8208,  0.1284,  0.0266],
         [ 0.8806, -0.3243,  0.2881,  ..., -0.3994, -0.1309,  0.7042],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]), tensor([[[ 0.3419,  0.0934,  0.1832,  ..., -0.4819,  0.0700,  0.2659],
         [ 0.2580,  0.0730,  0.0971,  ..., -0.6615, -0.1305,  0.3981],
         [ 0.8806, -0.3243,  0.2881,  ..., -0.3994, -0.1309,  0.7042],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]), tensor([[[ 0.3419,  0.0934,  0.1832,  ..., -0.4819,  0.0700,  0.2659],
   